## Bibliotecas

In [1]:
import cv2
import numpy as np
import random
import glob

## Detecção

In [2]:
def detectarPoligonos(frame):

    img_cinza = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    imagens_binarias = []
    for tresh in [100,180,200,110,225,215]:
        imagens_binarias.append(cv2.threshold(img_cinza, tresh, 255, 0)[1])
    
    deteccoes = []
    for img_bin in imagens_binarias:
        deteccoes += cv2.findContours(img_bin, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    deteccoes = [det for det in deteccoes if cv2.contourArea(det) < 1150.0 and cv2.contourArea(det) > 800.0 
                 and np.max(det[0].T[1])<int(0.40*frame.shape[0]) and np.max(det[0].T[1])>int(0.10*frame.shape[0])]
    
    for det in deteccoes:
        det = cv2.convexHull(det)
        quad = cv2.approxPolyDP(det, 0.02 * cv2.arcLength(det, True), True)
        if len(quad) >=4 and len(quad) <= 8:
            cv2.drawContours(frame, [quad], -1, (0,255,255), 2)
    
    return frame

## Rastreamento

In [3]:
rastreador = cv2.TrackerKCF_create()
video = cv2.VideoCapture("f1.mp4")
size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'MPEG')
out = cv2.VideoWriter('tracking.avi',fourcc, 36, size)

In [4]:
trackers = cv2.legacy.MultiTracker_create()
cap = cv2.VideoCapture("f1.mp4")
ok, frame = cap.read()

boxes = [cv2.selectROI('MultiTracker', frame), cv2.selectROI('MultiTracker', frame)]
colors = [(255, 0, 0), (0,0,255)]

for box in boxes:
    tracker = cv2.legacy.TrackerKCF_create()
    trackers.add(tracker, frame, box)

while cap.isOpened():

    ret, frame = cap.read()

    if  not ret:
        break

    (success, boxes) = trackers.update(frame)
    for i in range(len(boxes)):
        (x, y, w, h) = [int(v) for v in boxes[i]]
        cv2.rectangle(frame, (x, y), (x + w, y + h), colors[i], 2, 1)
        if i ==0:
            cv2.putText(frame, "PEREZ", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
        else:
            cv2.putText(frame, "LECLERC", (x,y-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    frame = detectarPoligonos(frame)
    out.write(frame)    
    
    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0XFF == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()